# Simple Code-Shortening Idioms #

A collection of some very simple techniques for performing common tasks, that hopefully you will find useful.

## Conditionally Deleting A Dictionary Entry ##

You may know that you can delete a dictionary entry with

    del dict[key]

but this raises a `KeyError` exception if `key` is not present in the dictionary. So how do you delete an entry _only if it is present_ ? This

    if key in dict :
        del dict[key]
    #end if

seems unwieldy. But there is a shorter way. The `dict` class defines [a pop method](https://docs.python.org/3/library/stdtypes.html#mapping-types-dict) that removes and returns an entry with a specified key; this call also raises `KeyError` if the key is not present, unless you specify a default value that is returned in this case. So, to delete an entry if present and avoid any error from its absence, pass any dummy default value (e.g. `None`) and ignore the result:

    dict.pop(key, None)

## Conditionally Looking Up A Dictionary Entry ##

(This one seems so obvious, yet I have seen code that was clearly written without knowing it could be done this way.)

If you try to access a dictionary entry `dict[key]` that does not exist, you will get a `KeyError` exception. If you want to get the entry if it is there, but do something else if it is not, I have seen code like

    if key in dict :
        val = dict[key]
        ... do something with val ...
    else :
        ... do something else ...
    #end if

There is a more direct way. The `dict` class defines a `get()` method, which returns a default value if the key is not present. You can specify your own default value, but if not, the default for the default is `None`. _E.g._

    val = dict.get(key)
    if val != None :
        ... do something with val ...
    else :
        ... do something else ...
    #end if


## Sorting & Removing Duplicates ##

If you have a list of items «items» (perhaps even being returned from an iterator), and you want to remove duplicate items and sort the rest into some kind of order, getting rid of the duplicates is easy by putting the items into a `set`:

    sorted(set(«items»))

You can specify a sort `key` argument to the `sorted()` function if you need more control over the sort order.

## Testing Mask Bits ##

You may not have immediately noticed from reading the Python language spec, but [binary bitwise operators have higher precedence than comparisons](https://docs.python.org/3/reference/expressions.html#binary-bitwise-operations). This differs from C and most other languages that follow C conventions. You may think this could easily trip up someone transliterating expressions from C or other such languages to Python, but I am not aware of any real-world situation where this could be a problem.

What it _does_ mean is that you can write bitmask tests with fewer parentheses. For example, in C you have to write things like

    («flags» & «mask») != 0
    («flags» & «mask1») == «mask2»

but in Python these parentheses are unnecessary:

    «flags» & «mask» != 0
    «flags» & «mask1» == «mask2»


## Loading A File Into Memory ##

Opening a file read-only and loading its entire contents into memory can be done with one line:

    string_contents = open(«filename», "rt").read()

or

    binary_contents = open(«filename», "rb").read()

But what about closing the file after reading? Ah, the nice thing is, CPython manages objects by reference-counting, so once the reference count of the `file` object reaches zero, it is automatically closed.


## Avoiding The `with`-Statement ##

`with`-statements have their uses, for example for managing memory allocated by some lower-level library that is not creating Python objects. But I often see them used with files and temporary directories, e.g.

    with open(«filename», «mode») as fileobj :
        ... do something with fileobj ...
    #end with

or

    with tempfile.TemporaryDirectory() as tempdir :
        ... create and manipulate various files in tempdir ...
    #end with

The aim is that the `with`-statement guarantees that, in any situation short of process deletion or complete runtime context meltdown, the file will be closed and the temporary directory (and its contents) cleaned up.

Trouble is, this is not necessarily a good thing.

For files opened read-only, this is unnecessary anyway, because as mentioned in the previous section, the file will be automatically closed when the last reference to the file object disappears.

For files opened for writing, I always have this concern about notifying the user as far as possible about any I/O errors. Because to fail silently could lead to data loss, where the user assumes that data has been saved, but it has not. So I always like to explicitly close such files:

    fileobj = open(«filename» «writemode»)
    .... fileobj.write(«data») ...
    fileobj.close()

That way any errors on flushing that last bufferful get a chance to be reported on the `close()` call. If the program never gets that far, then the user will not be misled by seeing a `with`-statement into assuming that any meaningful cleanup is going to be done.

For temporary directories, the `with`-statement guarantees as far as possible that the temporary directory will disappear, even if an exception occurs. Trouble is, if an exception occurs, the contents of that directory could be useful for debugging purposes. So, as messy as it may appear, I like to explicitly clean up the directory myself:

    tempdir = tempfile.mkdtemp()
    ... create and manipulate various files in tempdir ...
    shutil.rmtree(tempdir)

So now the temporary files only disappear if all goes well. If you see them piling up, then there is something wrong with your code!

## Avoiding Python Abortions 1: Conditional And Switch Expressions ##

Python’s conditional expression, introduced in Python 2.5,

    «true-part» if «cond» else «false-part»”

is truly appalling. With other “features” slavishly copied from C (like the use of “=” for assignment and “==” for comparison, instead of “:=” for the former and “=” for the latter as in the ALGOL-like languages), you’d think that adoption of one of C’s sensible ideas, namely

    «cond» ? «true-part» : «false-part»

would have been a no-brainer. It looks like the construct actually adopted was designed to make maximum use of existing reserved words, rather than make maximum sense.

Luckily, there are ways to avoid this construct in Python.

### Alternative Conditional Expressions ###

The obvious way to write a conditional expression is to use a boolean selector to index into a two-element array:

    («false-part», «true-part»)[«cond»]

However, while this may be adequate for simple cases, it does not give you short-cut evaluation. E.g. an expression like

    (0, x / y)[y != 0]

will fail with a `ZeroDivisionError` exception when y is 0, even though you want to return a different answer in that situation. But this is easily fixed, by putting the expressions into `lambda`s, and only evaluating the chosen one:

    (lambda : 0, lambda : x / y)[y != 0]()

Note the empty parentheses on the end, which cause the invocation of the chosen function.

### Switch-Expressions ###

The same sort of idea can be adapted to provide something that Python does not currently provide a special syntax for at all: switch-expressions. These were common among Algol-derivative languages, but are inexplicably missing from C and its derivatives. Among the Algol-like languages, you had both switch-expressions and switch-statements, which came in both labelled and unlabelled varieties. C has only the labelled form of switch-statement, and Python has no switch-statements at all. But both varieties of switch-expression are not hard to implement. Behold the unlabelled form:

    (
        lambda : case_0_expr,
        lambda : case_1_expr,
        lambda : case_2_expr,
        ...
    )[index]()

which evaluates _case_0_expr_ when `index` equals 0, _case_1_expr_ when it equals 1, and so on. Or you can have the form where each alternative is explicitly labelled with its selector value:

    {
        key1 : lambda : case_key1_expr,
        key2 : lambda : case_key2_expr,
        key3 : lambda : case_key3_expr,
        ...
    }[index]()

which evaluates _case_key1_expr_ when `index` equals `key1`, _case_key2_expr_ when `index` equals `key2`, and so on. Unlike in some of the Algols, the `key`_n_ expressions can be any expressions, since they are not restricted to being evaluable at compile time. And they can be of any valid type that Python will accept as dictionary keys, such as strings—they don’t have to be numeric!

But what happens if `index` happens not to match any of the `key`_n_ expressions? Python will raise a `KeyError` exception in this case. That may be what you want; but if you don’t, and you would rather a default value be returned in this case, that can be easily arranged:

    {
        key1 : lambda : case_key1_expr,
        key2 : lambda : case_key2_expr,
        key3 : lambda : case_key3_expr,
        ...
    }.get(index, default_expr)]()


## Avoiding Python Abortions 2: No Walruses Here ##

The “walrus” construct (a.k.a “assignment expressions”), introduced in Python 3.8, has been controversial. And the examples of it in use seem unconvincing. Particularly since much of the same effect can be achieved with an existing Python construct.

To illustrate what I mean, consider an [example or two](https://www.python.org/dev/peps/pep-0572/#examples) from the PEP supposedly demonstrating the benefits of the walrus.

From `datetime.py` in the standard library, original:

    s = _format_time(self._hour, self._minute, self._second, self._microsecond, timespec)
    tz = self._tzstr()
    if tz:
        s += tz
    return s

supposed improvement:

    s = _format_time(self._hour, self._minute, self._second, self._microsecond, timespec)
    if tz := self._tzstr():
        s += tz
    return s

My suggestion:

    return \
      (
            _format_time(self._hour, self._minute, self._second, self._microsecond, timespec)
        +
            (("", tz)[tz] for tz in (self._tzstr(),))[0]
      )

Simplifying list comprehensions: instead of

    results = [(x, y, x/y) for x in input_data if (y := f(x)) > 0]

how about

    results = [(x, y, x/y) for x in input_data for y in (f(x),) if y > 0]

and instead of

    stuff = [[y := f(x), x/y] for x in range(5)]

how about

    stuff = [[y, x/y] for x in range(5) for y in (f(x),)]

Anyway, you get the idea. And this works with older versions of Python, too!

## Do I Need To Mention This? ##

Have you seen code that looks like

    try:
        f = open(filename)
    except:
        print("couldn’t open %s" % filename)
        sys.exit(-1)
    ... process f ...

I have, more than once. What exactly is this trying to achieve? It catches an error on opening the file; yet, instead of explaining to the user what the error might be, it simply prints a generic message and exits. Far better not to bother catching the exception at all:

    f = open(filename)
    ... process f ...

That way, if there is any problem opening the file, the user sees an informative message, explaining exactly what the problem is (e.g. file not found, or no access permission), together with a traceback explaining where it is happening. Isn’t that better?

I remember many years ago reading a collection of programming adages, among which was

> **Never test for an error condition you don’t know how to handle.**

It wasn’t meant seriously, yet it seems somehow appropriate here ...